#Open Library Analysis - Big Data Computing Project
####Graph's Connected Components vs. k-means Clusters

##Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

##Data Aquisition

short dataset: 50000 entries

In [0]:
%sh wget -P /tmp https://raw.githubusercontent.com/attennig/BDC_datasets/main/books_short.csv

In [0]:
dbutils.fs.mv("file:/tmp/books_short.csv", "dbfs:/bdc-2020-21/datasets/books_short.csv")

long dataset: 22.589.356 entries 

remember to download it only if it is necessary

In [0]:
%sh wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G" -O /tmp/books_long.csv.bz2 && rm -rf /tmp/cookies.txt

In [0]:
dbutils.fs.ls("file:/tmp")

In [0]:
dbutils.fs.mv("file:/tmp/books_long.csv.bz2", "dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
dbutils.fs.ls("dbfs:/bdc-2020-21/datasets")

In [0]:
# Read dataset file into a Spark Dataframe
books_df = spark.read.load("dbfs:/bdc-2020-21/datasets/books_short.csv", 
                         format="csv", 
                         sep=";", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
tot_entries = books_df.count();
print("The shape of the dataset is {:d} rows by {:d} columns".format(tot_entries, len(books_df.columns)))

The shape of the dataset is 50000 rows by 17 columns

##Data Cleaning

In [0]:
#eliminating entries with missing values or columns
columns_to_drop = []
for c in books_df.columns:
  if books_df.where(col(c).isNull()).count()/tot_entries > 0.7:
    #remove col
    columns_to_drop += [c]

In [0]:
columns_to_drop

Out[5]: ['subtitle',
 'translated_titles',
 'description',
 'dewey_number',
 'lc_classifications',
 'first_sentence',
 'original_languages ',
 'other_titles',
 'first_publish_date',
 'links',
 'notes',
 'cover_edition',
 'covers']

In [0]:
books_df = books_df.drop(*columns_to_drop)

In [0]:
assert "title" in books_df.columns and "key" in books_df.columns
books_df = books_df.dropna(how="any", subset=["key", "title"])
books_df = books_df.dropDuplicates(['key'])
books_df = books_df.dropDuplicates(['title'])

In [0]:
assert "subjects" in books_df.columns and "authors" in books_df.columns

books_df = books_df.na.fill({'subjects': 'unknown', 'authors': 'unknown'})

In [0]:
books_df.describe().show()

+-------+-----------+--------------------+-------------+--------------------+
summary| key| title| authors| subjects|
+-------+-----------+--------------------+-------------+--------------------+
 count| 49626| 49626| 49626| 49626|
 mean| null| 1501.6705882352942| null| null|
 stddev| null| 737.069345249063| null| null|
 min|OL10000079W| - die Hoffnung m...|['OL100070A']|["Accord Canada-O...|
 max|OL23534178W|︠T︡siviliza︠t︡si︠...| unknown| unknown|
+-------+-----------+--------------------+-------------+--------------------+

In [0]:
# if nltk not found
!python -m pip install nltk

!python -m pip install --upgrade pip

!python -m nltk.downloader all

In [0]:
# From Document_Clustering.ipynb
def clean_text(df, column_name):
    """ 
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline

    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE "LIBRARIES" TAB IN THE MENU

    # Text preprocessing pipeline
    # 1. Text cleaning
    # 1.a Case normalization
    lower_case_df = df.select(lower(col(column_name)).alias(column_name))
    # 1.b Trimming
    trimmed_df = lower_case_df.select(trim(col(column_name)).alias(column_name))
    # 1.c Filter out punctuation symbols
    no_punct_df = trimmed_df.select((regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name))
    # 1.d Filter out any internal extra whitespace
    cleaned_df = no_punct_df.select(trim(regexp_replace(col(column_name), " +", " ")).alias(column_name))
    # 2. Tokenization (split text into tokens)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_df).cache()
    # 3. Stopwords removal
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    terms_df = stopwords_remover.transform(tokens_df).cache()
    # 4. Stemming (Snowball stemmer)
    stemmer = SnowballStemmer(language="english")
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    terms_stemmed_df = terms_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()

    return terms_stemmed_df

In [0]:
clean_title_df = clean_text(books_df, "title")

In [0]:
clean_subjects_df = clean_text(books_df, "subjects")

##Feature Engineering

In this section we will use nlp techniques to get numerical vectors that represent text-based features

In [0]:
# TF-IDF or Word2Vec from Document_Clustering.ipynb 

##Graph

In [0]:
# guardare notebook pagerank
# creare df con colonna id per i nodi
# creare df con colonne src e dst per gli archi
# def sim(u,v)
# gli archi vanno agginti bidirezionali per le entries che rispettalo la condizione sim(u,v) > eps
# crea grafo
# ottieni CC del grafo


##Clustering

In [0]:
# from Document_Clustering.ipynb  e l'altro del clustering
# k = nCC , kmeans

##Evaluation